In [1]:
%matplotlib inline 
import pandas as pd
import numpy as np
import xarray as xr
import os
import glob
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy.stats import norm
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

import seaborn as sns
sns.set_style('white')
sns.set_context("notebook") #[poster, paper, talk, notebook]

### Add Lidar Data

In [10]:
fname = '/Users/meganmason491/Documents/research/sierra/data/compiled_SUPERsnow.nc' #3m BSU
# fname = '/Users/megmason/research/sierra/data/compiled_SUPERsnow_20m.nc' #20m mac #CAREFUL THIS IS 20m DATA

#open data
ds = xr.open_dataset(fname, chunks={'time':1,'x':1000,'y':1000})
ds['snow'] = ds.snow / 10
ds.attrs['units'] = 'cm'
ds=ds.astype(np.int16, copy=False)
ds.close()

### Add Terrain Data

In [4]:
fpath = '/Users/meganmason491/Documents/research/sierra/data/terrain/*flipped*.nc' #BSU
# fpath = '' #mac

terrain=xr.open_mfdataset(fpath, 
                          concat_dim=None, 
                          combine='nested', 
                          chunks={'x':1000, 'y':1000}, 
                          parallel=True) 
terrain = terrain.where(ds.mask==1)
terrain.close()

In [18]:
ds

<xarray.Dataset>
Dimensions:  (time: 51, x: 17567, y: 16780)
Coordinates:
  * y        (y) float64 4.23e+06 4.23e+06 4.23e+06 ... 4.179e+06 4.179e+06
  * x        (x) float64 2.54e+05 2.54e+05 2.54e+05 ... 3.067e+05 3.067e+05
  * time     (time) datetime64[ns] 2013-04-03 2013-04-29 ... 2018-05-28
    mask     (y, x) uint8 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    dem      (y, x) float32 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    veg      (y, x) float32 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    srf      (y, x) float32 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
Data variables:
    snow     (time, y, x) int16 dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>

In [19]:
#all elevations above 2000m
d = ds.where(ds.dem>=3000., drop=True)


In [14]:
d.isel(time=31).snow.plot.imshow(x='x',y='y',col='time')
d.isel(time=31).snow.plot.imshow(x='x',y='y',col='time')

//anaconda3/envs/data_analysis/lib/python3.6/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in greater_equal
  return func(*args2)


ValueError: IndexVariable objects must be 1-dimensional

In [22]:
d.dem.min().values

array(1078.0702, dtype=float32)

In [17]:
ds.dem.min().values

//anaconda3/envs/data_analysis/lib/python3.6/site-packages/dask/compatibility.py:107: RuntimeWarning: All-NaN slice encountered
  return func(*args, **kwargs)
//anaconda3/envs/data_analysis/lib/python3.6/site-packages/toolz/functoolz.py:488: RuntimeWarning: All-NaN slice encountered
  ret = f(ret)


array(1078.0702, dtype=float32)

In [23]:
pix=501
tum = ds.sel(x=slice(293484.-pix, 293484.+pix), y=slice(4194702.+pix, 4194701.-pix))

In [49]:
print('min elevation', tum.dem.min().values)
print('max elevation:', tum.dem.max().values)
tum

min elevation 2615.869
max elevation: 2844.115


<xarray.Dataset>
Dimensions:  (time: 51, x: 335, y: 335)
Coordinates:
  * y        (y) float64 4.195e+06 4.195e+06 4.195e+06 ... 4.194e+06 4.194e+06
  * x        (x) float64 2.93e+05 2.93e+05 2.93e+05 ... 2.94e+05 2.94e+05
  * time     (time) datetime64[ns] 2013-04-03 2013-04-29 ... 2018-05-28
    mask     (y, x) uint8 1 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1
    dem      (y, x) float32 2652.318 2652.318 2652.318 ... 2630.2932 2630.2932
    veg      (y, x) float32 4.6484985 3.6049805 1.3543701 ... 5.145752 4.1503906
    srf      (y, x) float32 2.25 2.5 2.75 3.0 3.0 2.75 ... 2.0 2.0 2.0 2.0 2.0
Data variables:
    snow     (time, y, x) float64 dask.array<chunksize=(1, 335, 9), meta=np.ndarray>

In [47]:
foo = tum.where(tum.dem>2800.)
foo.dem.min().values

array(2615.869, dtype=float32)